## Vision-Python Demonstration Workbook: Vision Language Advanced

*Objective*: To illustrate access to a vision service that delivers standard python-ready json structures with the longer-term goal of demonstrating access to client FAST environments.


### Audience

This notebook contains some advanced Vision coding examples that focus on creating, manipulating, and returning JSON structures from Vision. The audience for these examples is expected to have basic familiarity with the Vision language and with python.  [Introductory Example](https://github.com/LeslieNewman/Vi-Py/blob/main/py/VisionExpressionsStoryBoard.ipynb) are available as well.

#### Notes

* This notebook provides a number of examples that fetch data from a Vision federated database utilizing an open-source node-js package talking to vision.  This is a work-in-progress designed to demonstrate concepts.  Only FactSet internals will have access to this database.  All of the components demonstrated here are designed to work with client Vision environments as well.

* You will need to set an environment variable to tell python where the Vision Server is listening.  Use the python _os.environm_ function to set the server name.


### Getting Started

To begin, you will need to import the Vision Connection (vconnect) module.  __[Get the <i>vconnect</i> module here](https://github.com/LeslieNewman/Vi-Py/blob/main/py/vconnect.py)__ .  Store this file locally in your normal work area.  The examples will be using several functions from this module:

* *vc.runvget* is used to submit a Vision expression as a *get* url.
* *vc.runvision* is used to run a Vision query and return the results as text
* *vc.runvisionJSON* is used to run a Vision query and return the results as a JSON dictionary.  

The examples below also use the *DataFrame* class from the *pandas* module, so you probably want to import that as well.

You will need to use the os.environ function to identify the location of your Vision Server.


In [1]:
import pandas as pd
import vconnect

vc = vconnect.VCconnection()
vc.setUrl('http://visdevdb01.cts.fast-clientenv-aws.dev.us-east-1.aws.fdscloud.io:10010/vision/api')

### Waking up the Vision Service

This demo is leveraging a non-production service running on a dev box.  At any time, the service may be in one of the following states:

1. Available
1. Needing Restart
1. Dead

The following code can be run to determine which state the service is in:

>`
  vc.runvision ('2 + 2')
>`

If the service is **available**, you will see an appropriate response.  If the service is **needing restart**, you will get an error message that includes the text *RemoteDisconnected* after a minute or so.  If you see this message, you should run the code again which should restart the service for you.  This step may take a minute or two to finish.  If you get another error message, the service is most like **dead** and will need manual intervention on the unix account.


In [2]:
#-- wake up the Vision service
print (vc.runvision('2 + 2'))

     4.00



## Building Your Own JSON Structure in Vision

The _getArrayFrom:for:over:_ message defined in the [Introductory Examples](https://github.com/LeslieNewman/Vi-Py/blob/main/py/VisionExpressionsStoryBoard.ipynb) notebook enables you to run a vision block designed to return a simple list of named values. There are a number of new methods defined at the Vision class **JS** that enable you to  define your own complex JSON structure and return the object to your client, in this case, your python program.  The values can be integers, doubles, strings, other objects, or arrays.  

#### Messages defined at **JS** 

* *getConnection*          --  return a reference to an external connection object
* *newObject*              --  create an object that is rendered as an external dictionary object
* *newArrayFrom: list .*   --  converts a vision list to an external array object

* *returnJSON: obj .*   --  return the supplied obj to the client

#### Messages define at **JS newObject**

* *set: key to val .*  --  sent to an external object to set supplied key to supplied val

### Notes

* When you run the vision code from the python environment, the external object referenced by _getConnection_ represents your python client.  When you run this same code from within an interactive _batchvision_ session, an external object will be created within that batchvision.




In [3]:
#-- Sample Query: Create an object that contains information about an account


#--  Create the Vision Code to Run
query = '''
!connectObject <- JS getConnection ;
!acct <- Account findId: "SP50" ;

#-- Create a JS object with some key/value pairs
!data <- connectObject newObject
  set: "id" to: acct id .
  set: "name" to: acct name .
  set: "count" to: acct holdings count .
  set: "date" to: acct holdingsDate asInteger ;

#-- Return the object as JSON 
connectObject returnJSON: data

'''

#-- Call Vision to run the expression, returning a JS object
response = vc.runvisionJSON(query)

#--- Print the response and its type
print ("the response is of type ", type(response))
print (response)

#-- When we use the JSON call, the response is converted to a python dictionary
#-- so we can access elements directly
print ("We can access pieces of the response:")
print ("Id is: " , response['id'], " and name is: ", response['name'])

the response is of type  <class 'dict'>
{'id': 'SP50', 'name': 'S&P 500', 'count': 505, 'date': 20210202}
We can access pieces of the response:
Id is:  SP50  and name is:  S&P 500


In [4]:
#-- Expanding our example to return a named array containing the holdings

query = '''
!connectObject <- JS getConnection ;

!acct <- Account findId: "r.1000" ;

#--  Create a JS object with some key/value pairs
!data <- connectObject newObject 
   set: "id" to: acct id .
   set: "name" to: acct name .
   set: "count" to: acct holdings count .
   set: "date" to: acct holdingsDate asInteger ;
   
#-- create an array of JS holding objects
!holdObjects <-
acct holdings sortDown: [ percentOfPort ] . first: 5 . 
 send: [ ^my connectObject newObject 
         set: "date" to: ^date asString .
         set: "acctId" to: account code .
         set: "secId" to: security code .
         set: "weight" to: percentOfPort .
       ] ;
       
#-- add a key-value pair to data object that references this vision list as an array
data set: "holdings" to: (connectObject newArrayFrom: holdObjects ) ;   
   
#-- Return the object as JSON   
 connectObject returnJSON: data

'''

#--  Call Vision to run the expression
getHoldings = vc.runvisionJSON(query)

print (getHoldings)

{'id': 'R.1000', 'name': 'Russell 1000', 'count': 1013, 'date': 20210202, 'holdings': [{'date': '2/3/2021', 'acctId': 'R.1000', 'secId': '03783310', 'weight': 5.999659246152214}, {'date': '2/3/2021', 'acctId': 'R.1000', 'secId': '59491810', 'weight': 4.939494591718715}, {'date': '2/3/2021', 'acctId': 'R.1000', 'secId': '02313510', 'weight': 3.947050349395297}, {'date': '2/3/2021', 'acctId': 'R.1000', 'secId': '88160R10', 'weight': 1.8175740363864614}, {'date': '2/3/2021', 'acctId': 'R.1000', 'secId': '30303M10', 'weight': 1.7662309209958527}]}


In [5]:
#-- By default, we are printing the entire js object.  DataFrames help here

print("Holdings for: " + getHoldings['id'] + "  " + getHoldings['name'])
pd.DataFrame(getHoldings['holdings'])

Holdings for: R.1000  Russell 1000


,acctId,date,secId,weight
0,R.1000,2/3/2021,03783310,5.999659
1,R.1000,2/3/2021,59491810,4.939495
2,R.1000,2/3/2021,02313510,3.947050
3,R.1000,2/3/2021,88160R10,1.817574
4,R.1000,2/3/2021,30303M10,1.766231


## Packaged Applications with the Applay Class

The **Applay** class has been created to support building structured applications in Vision that return JSON dictionary objects.   Applications are defined at the class **Applay Lib** as methods that can read inputs from a supplied JSON dictionary object.

Within your method you can use the following messages:

* *getNewObject*  -- returns a js dictionary object that responds to the *set: key to: val* message
* *getNewArrayFrom: list*  -- returns an array from a list of js dictionary objects
* *responseObject*  --  the js dictionary object that will be returned to your calling program
* *updateStatusWithSuccess*  -- marks the status flag in the retuned js dictionary object
* *postErrorWithMessage: errorString* -- returns the error message supplied

This [Jupyter Notebook](https://github.com/LeslieNewman/Vi-Py/blob/main/py/VisionStoryBoard.ipynb) describes a number of prepackaged applications.


In [6]:
#---  Create an Application called Tester
#---    params = acctId, date, num
app = '''

Applay Lib defineMethod: [ | Tester |

!id <-   ^self getParam: "acctId" ;
!date <- ^self getParam: "date" . asDate else: ^date ;
!max <-  ^self getParam: "num" . asInteger  ;
!entity <- GlobalWorkspace Account findId: id ;
entity isntDefault
ifTrue:
  [ !holdings <- date evaluate: [ entity holdings ] ;
    max isntNA
      ifTrue: [ :holdings <- holdings sortDown: [ percentOfPort ] . first: max ] ;
    ^self responseObject
        set: "id" to: entity id .
        set: "name" to: entity name .
       ;
     !constits <- holdings
        send: [ ^my getNewObject
                    set: "acctId" to: account code .
                    set: "date" to: date formatUsingShortName .
                    set: "secId"
                     to: security descriptiveData factsetId .
                    set: "name" to: security name .
                    set: "weight" to: percentOfPort .
                    set: "mval" to: totalMarketValue .
                ] ;
    ^self responseObject
         set: "constituents" to: (^self getNewArrayFrom: constits);
    ^self updateStatusWithSuccess ;
    ]
ifFalse:
   [ ^self postErrorWithMessage: ("Id " concat: id . concat: " not found.")
   ] ;

^self
] ;

'''
#-- Define the Method and confirm that it is there
vc.runvision(app)

#--  Run the Application
params = {'acctId' : "SP50", 'num' : 3}
response = vc.runapp ("Tester", params)
print( "Status: ", response['status'])
print ("Response:", type(response))
print (response)

pd.DataFrame(response['response']['constituents'])

#--- Run it with Bad inputs
#print (vc.runapp("Tester", {'acctId' : "badId"}))

Status:  {'code': 0, 'message': 'Success'}
Response: <class 'dict'>
{'appName': 'Tester', 'status': {'code': 0, 'message': 'Success'}, 'response': {'id': 'SP50', 'name': 'S&P 500', 'constituents': [{'acctId': '00000117', 'date': ' 2-Feb-2021', 'secId': '03783310', 'name': 'Apple Inc.', 'weight': 6.681103904487748, 'mval': 2157368983751.7998}, {'acctId': '00000117', 'date': ' 2-Feb-2021', 'secId': '59491810', 'name': 'Microsoft Corp', 'weight': 5.607867374079184, 'mval': 1810814396960}, {'acctId': '00000117', 'date': ' 2-Feb-2021', 'secId': '02313510', 'name': 'Amazon.com Inc', 'weight': 4.4642413728480586, 'mval': 1441530623000}]}, 'snfs': []}


,acctId,date,mval,name,secId,weight
0,00000117,2-Feb-2021,2.157369e+12,Apple Inc.,03783310,6.681104
1,00000117,2-Feb-2021,1.810814e+12,Microsoft Corp,59491810,5.607867
2,00000117,2-Feb-2021,1.441531e+12,Amazon.com Inc,02313510,4.464241
